In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from procedures import create_binance_bot, create_binance_bot_spot, make_get_filepath
from pure_funcs import get_template_live_config, ts_to_date, get_daily_from_income
from njit_funcs import round_dynamic
from time import time, sleep
import os
import pandas as pd
import asyncio
import numpy as np
import matplotlib.pyplot as plt

using numba


In [3]:
plt.rcParams["figure.figsize"] = [21, 13]
pd.set_option("display.precision", 10)

In [6]:
config = get_template_live_config()
config["symbol"] = "XMRUSDT"  # dummy symbol
config["exchange"] = "bitget"
config["market_type"] = "futures"
config["user"] = "bitget_01"
try:
    await bot.session.close()
except:
    pass
if config["market_type"] == "spot":
    bot = await create_binance_bot_spot(config)
else:
    bot = await create_binance_bot(config)

linear perpetual
initiating position, open orders, fills, exchange config, order book, and emas...


ERROR:root:error fetching open orders string indices must be integers
Traceback (most recent call last):
  File "C:\Users\Nitropc\Projects\passivbot\exchanges\binance.py", line 304, in fetch_open_orders
    return [
  File "C:\Users\Nitropc\Projects\passivbot\exchanges\binance.py", line 306, in <listcomp>
    "order_id": int(e["orderId"]),
TypeError: string indices must be integers
ERROR:root:error with update open orders 'bool' object is not iterable
Traceback (most recent call last):
  File "C:\Users\Nitropc\Projects\passivbot\passivbot.py", line 360, in update_open_orders
    open_orders = [x for x in open_orders if x["symbol"] == self.symbol]
TypeError: 'bool' object is not iterable


returned: {'code': -2014, 'msg': 'API-key format invalid.'}


ERROR:root:error fetching pos or balance 0
Traceback (most recent call last):
  File "C:\Users\Nitropc\Projects\passivbot\exchanges\binance.py", line 337, in fetch_position
    assert all(
  File "C:\Users\Nitropc\Projects\passivbot\exchanges\binance.py", line 338, in <genexpr>
    key in positions[0] for key in ["symbol", "positionAmt", "entryPrice"]
KeyError: 0
ERROR:root:error with update position 'NoneType' object is not subscriptable


2023-10-19T00:21:11  {'code': -2014, 'msg': 'API-key format invalid.'} 
returned: {'code': -2014, 'msg': 'API-key format invalid.'}
returned: {'code': -2014, 'msg': 'API-key format invalid.'}
error fetching fills string indices must be integers
error fetching latest fills string indices must be integers
returned: {'code': -2014, 'msg': 'API-key format invalid.'}


Traceback (most recent call last):
  File "C:\Users\Nitropc\Projects\passivbot\passivbot.py", line 428, in update_position
    position["wallet_balance"] = self.adjust_wallet_balance(position["wallet_balance"])
TypeError: 'NoneType' object is not subscriptable
Traceback (most recent call last):
  File "C:\Users\Nitropc\Projects\passivbot\exchanges\binance.py", line 560, in fetch_fills
    fills = [
  File "C:\Users\Nitropc\Projects\passivbot\exchanges\binance.py", line 562, in <listcomp>
    "symbol": x["symbol"],
TypeError: string indices must be integers
Traceback (most recent call last):
  File "C:\Users\Nitropc\Projects\passivbot\exchanges\binance.py", line 505, in fetch_latest_fills
    fills += [
  File "C:\Users\Nitropc\Projects\passivbot\exchanges\binance.py", line 507, in <listcomp>
    "order_id": elm["orderId"],
TypeError: string indices must be integers


2023-10-19T00:21:11  {'code': -2014, 'msg': 'API-key format invalid.'} 
2023-10-19T00:21:11  {'code': -2014, 'msg': 'API-key format invalid.'} 
done


ERROR:root:error fetching pos or balance 0
Traceback (most recent call last):
  File "C:\Users\Nitropc\Projects\passivbot\exchanges\binance.py", line 337, in fetch_position
    assert all(
  File "C:\Users\Nitropc\Projects\passivbot\exchanges\binance.py", line 338, in <genexpr>
    key in positions[0] for key in ["symbol", "positionAmt", "entryPrice"]
KeyError: 0
ERROR:root:error with update position 'NoneType' object is not subscriptable


returned: {'code': -2014, 'msg': 'API-key format invalid.'}
returned: {'code': -2014, 'msg': 'API-key format invalid.'}


Traceback (most recent call last):
  File "C:\Users\Nitropc\Projects\passivbot\passivbot.py", line 428, in update_position
    position["wallet_balance"] = self.adjust_wallet_balance(position["wallet_balance"])
TypeError: 'NoneType' object is not subscriptable


In [7]:
info = await bot.public_get(bot.endpoints["exchange_info"])
csvs = {}
syms = [x["symbol"] for x in info["symbols"] if x["symbol"].endswith("USDT")]
n = 7
interval = "5m"
while True:
    if not syms:
        break
    subset = syms[:n]
    syms = syms[n:]
    res = await asyncio.gather(*[bot.fetch_ohlcvs(symbol=s, interval=interval) for s in subset])
    for s, r in zip(subset, res):
        csvs[s] = pd.DataFrame(r)
    print(subset, len(syms))

['BTCUSDT', 'ETHUSDT', 'BCHUSDT', 'XRPUSDT', 'EOSUSDT', 'LTCUSDT', 'TRXUSDT'] 213
['ETCUSDT', 'LINKUSDT', 'XLMUSDT', 'ADAUSDT', 'XMRUSDT', 'DASHUSDT', 'ZECUSDT'] 206
['XTZUSDT', 'BNBUSDT', 'ATOMUSDT', 'ONTUSDT', 'IOTAUSDT', 'BATUSDT', 'VETUSDT'] 199
['NEOUSDT', 'QTUMUSDT', 'IOSTUSDT', 'THETAUSDT', 'ALGOUSDT', 'ZILUSDT', 'KNCUSDT'] 192
['ZRXUSDT', 'COMPUSDT', 'OMGUSDT', 'DOGEUSDT', 'SXPUSDT', 'KAVAUSDT', 'BANDUSDT'] 185
['RLCUSDT', 'WAVESUSDT', 'MKRUSDT', 'SNXUSDT', 'DOTUSDT', 'DEFIUSDT', 'YFIUSDT'] 178
['BALUSDT', 'CRVUSDT', 'TRBUSDT', 'RUNEUSDT', 'SUSHIUSDT', 'SRMUSDT', 'EGLDUSDT'] 171
['SOLUSDT', 'ICXUSDT', 'STORJUSDT', 'BLZUSDT', 'UNIUSDT', 'AVAXUSDT', 'FTMUSDT'] 164
['HNTUSDT', 'ENJUSDT', 'FLMUSDT', 'TOMOUSDT', 'RENUSDT', 'KSMUSDT', 'NEARUSDT'] 157
['AAVEUSDT', 'FILUSDT', 'RSRUSDT', 'LRCUSDT', 'MATICUSDT', 'OCEANUSDT', 'CVCUSDT'] 150
['BELUSDT', 'CTKUSDT', 'AXSUSDT', 'ALPHAUSDT', 'ZENUSDT', 'SKLUSDT', 'GRTUSDT'] 143
['1INCHUSDT', 'CHZUSDT', 'SANDUSDT', 'ANKRUSDT', 'BTSUSDT', 'LITUS

In [ ]:
def correlation(x, y):
    x_mean = x.mean()
    y_mean = y.mean()
    a = x - x_mean
    b = y - y_mean
    return ((a * b).sum()) / (((a ** 2).sum() * (b ** 2).sum()) ** 0.5)

In [ ]:
n_symbols = 30
vols = [(csvs[s].volume * csvs[s].close).sum() for s in csvs]
minvol = sorted(vols)[-n_symbols]
ss = [s for s in csvs if (csvs[s].volume * csvs[s].close).sum() >= minvol]
corrs = []
corrs_dict = {}
done = set()
for s0 in sorted(ss):
    for s1 in sorted(ss):
        if s0 == s1:
            continue
        key = tuple(sorted([s0, s1]))
        if key in done:
            continue
        done.add(key)
        x = csvs[s0].close
        y = csvs[s1].close
        corr = correlation(x, y)
        print(f"\r{s0} {s1} {corr:.4f}      ", end=" ")
        corrs.append((corr,) + key)

corrs_dict = {}
corrs_dict_abs = {}
for e in corrs:
    if e[1] not in corrs_dict:
        corrs_dict[e[1]] = {e[2]: e[0]}
        corrs_dict_abs[e[1]] = {e[2]: abs(e[0])}

    else:
        corrs_dict[e[1]][e[2]] = e[0]
        corrs_dict_abs[e[1]][e[2]] = abs(e[0])

    if e[2] not in corrs_dict:
        corrs_dict[e[2]] = {e[1]: e[0]}
        corrs_dict_abs[e[2]] = {e[1]: abs(e[0])}

    else:
        corrs_dict[e[2]][e[1]] = e[0]
        corrs_dict_abs[e[2]][e[1]] = abs(e[0])

In [ ]:
# sum correlation of each symbol to every other symbol
corr_sorted = sorted([(sum(corrs_dict[s].values()), s) for s in corrs_dict])
corr_sorted

In [ ]:
# mean of all symbol
mean_prices = pd.concat([e.close / e.close.iloc[0] for e in csvs.values()], axis=1).mean(axis=1)

# compare individual symbol to mean
symbol = "SXPUSDT"
# symbol = corr_sorted[0][1]  # least correlated symbol
mean_prices.rename("mean_prices").plot()
(csvs[symbol].close / csvs[symbol].close.iloc[0]).rename(symbol).plot()
plt.legend()

In [ ]:
# compare two symbols
symbol0 = "BNBUSDT"
symbol1 = "LINKUSDT"
(csvs[symbol0].close / csvs[symbol0].close.iloc[0]).rename(symbol0).plot()
(csvs[symbol1].close / csvs[symbol1].close.iloc[0]).rename(symbol1).plot()
plt.legend()